In [ ]:
#0작품명: <작품명>
import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'

# 변환할 단어 목록 (변환 후 단어 : [변환 전 패턴들])
special_chars = ['＇', r'\"', r'\(', r'\)', r'\[', r'\]', r'\{', r'\}', r'\<', r'\>', '《', '》', '｢', '｣', '󰡔', '󰡕', '【', '】']

# 변환 후 단어들 리스트 및 변환 전 단어 규칙
terms = {
    "Q씨에게": [r"Q\s*씨\s*에\s*게"],
    "가을에_온_여인": [r"가\s*을\s*에\s*온\s*여\s*인"],
    "김약국의_딸들": [r"김\s*藥\s*局\s*의\s*딸\s*들", r"金\s*약\s*국\s*의\s*딸\s*들", r"김\s*약\s*국\s*의\s*딸\s*들", r"김\s*약\s*국\s*의\s*딸"],
    "문학을_사랑하는_젊은이들에게": [r"문\s*학\s*을\s*사\s*랑\s*하\s*는\s*젊\s*은\s*이\s*들\s*에\s*게", 
                                r"문\s*학\s*을\s*지\s*망\s*하\s*는\s*젊\s*은\s*이\s*들\s*에\s*게"],
    "성녀와_마녀": [r"성\s*녀\s*와\s*마\s*녀", r"마\s*녀\s*와\s*성\s*녀"],
    "시장과_전장": [r"시\s*장\s*과\s*전\s*장", r"전\s*장\s*과\s*시\s*장"],
    "신교수의_부인": [r"신\s*교\s*수\s*의\s*부\s*인", r"申\s*교\s*수\s*의\s*부\s*인"],
    "애가": [r"애\s*가", r"연\s*가"],
    "어느_오후의_결정": [r"어\s*느\s*오\s*후\s*의\s*결\s*정", r"어\s*느\s*정\s*오\s*의\s*결\s*정"],
    "영원한_반려": [r"영\s*원\s*한\s*반\s*려", r"영\s*원\s*의\s*반\s*려"],
    "암흑시대": [r"暗黑시대"], "설화": [r"雪花"], "노을_진_들녘": [r"놀\s*진\s*들\s*녁"]
}

# 추가 작품명들 변환 규칙 생성 (기존 규칙)
additional_terms = [
"거리의_악사", "겨울비", "계산", "군식구", "귀족", "그_형제의_연인들", "기다리는_불안", "까치설", "꿈꾸는_자가_창조한다", "나비야_청산가자",
"나비와_엉겅퀴", "내_마음은_호수", "노을_진_들녘", "녹지대", "눈먼_식솔", "단층", "도선장", "도시의_고양이들", "도표없는_길", "돌아온_고양이",
"돌아온_아이", "만리장성의_나라", "만화_토지", "목련_밑", "못_떠나는_배", "밀고자", "반딧불", "뱁새족", "버리고_갈_것만_남아서_참_홀가분하다", "벽지",
"불신시대", "비는_내린다", "새벽의_합창", "생명의_아픔", "속단", "시경소화", "시장과_전장", "신교수의_부인", "쌍두아", "암흑시대",
"암흑의_사자", "애가", "어느_오후의_결정", "어머니", "영주와_고양이", "옛날_이야기", "옛날의_그집", "외곽지대", "우리들의_시간", "우화",
"원주통신", "은하", "은하수", "인간", "일본산고", "재귀열", "재혼의_조건", "전도", "죄인들의_숙제", "집",
"창", "창작실기론", "청소년_토지", "타인들", "토지", "파시", "평면도", "표류도", "푸른_운하", "풍경A",
"풍경B", "하루", "해동여관의_미나", "호수", "환상의_시기", "훈향", "흑백_콤비의_구두", "흑흑백백", "희오의_바다", "사랑섬_할머니",

]

# 추가 작품명들에 대해 기본 패턴 생성
for term in additional_terms:
    terms[term] = [r"\s*".join(term.replace('_', ''))]

# 변환 규칙 딕셔너리 생성
transformations = {}
for replacement, patterns in terms.items():
    full_patterns = []
    for pattern in patterns:
        for char1 in special_chars:
            for char2 in special_chars:
                full_patterns.append(rf"{char1}\s*{pattern}\s*{char2}")
    transformations[replacement] = full_patterns

# 변환된 단어를 기록하기 위한 딕셔너리
changes = {}

# 변환 함수 정의
def replace_and_log(original_text, pattern, replacement, changes_dict):
    # 변환 후 단어에 <>를 씌워줌
    modified_replacement = f"<{replacement}>"
    
    # 패턴에 매칭되는 모든 부분을 찾아서
    matches = re.findall(pattern, original_text)
    # 매칭된 부분이 있다면 변환하고 딕셔너리에 추가
    if matches:
        for match in matches:
            changes_dict[match] = modified_replacement
        # 텍스트에서 패턴을 변환
        modified_text = re.sub(pattern, modified_replacement, original_text)
        return modified_text
    return original_text

# 변환 대상 컬럼 설정
target_columns = ['title', 'sub-title', 'content']

# 모든 엑셀 파일을 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_excel(file_path)  # 엑셀 파일 읽기

        # 지정된 컬럼만 변환 진행
        for column in target_columns:
            if column in df.columns:
                print(f"  Processing column: {column}")
                for replacement, patterns in transformations.items():
                    for pattern in patterns:
                        df[column] = df[column].apply(
                            lambda x: replace_and_log(x, pattern, replacement, changes) if isinstance(x, str) else x
                        )
        
        # 변환된 데이터 저장
        output_path = os.path.join(folder_path, f'0_{filename}')
        df.to_excel(output_path, index=False)  # 변환된 파일을 새로운 이름으로 저장
        print(f"  Saved processed file: {output_path}")

# 변환된 단어 목록 출력
print("\n변환된 단어 목록:")
for original, transformed in changes.items():
    print(f"'{original}' -> '{transformed}'")


In [ ]:
#1인물명

import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'

# 변환할 단어 목록 (변환 후 단어 : [변환 전 패턴들])
terms = {
    "김동리": [rf"김\s*동\s*이"],
    "김영주": [rf"김\s*령\s*주"],
    "유종호": [rf"류\s*종\s*호"],
    "이어령": [rf"이\s*어\s*녕"],
    "박경리": [rf"박\s*경\s*이"],
}

# 추가 명들 변환 규칙 생성 (기존 규칙)
additional_terms = [
    "강만길", "김동리", "김영주", "김지하", "김치수", "김형국", "유종호", "이어령", "이종환", "박경리",
    "박금이", "박완서", "박화성", "백낙청", "백철",   "전광용", "정인영", "정현기", "최일남",
]

# 추가 명들에 대해 기본 패턴 생성
for term in additional_terms:
    if term not in terms:
        pattern = rf"{r'\s*'.join(term)}"
        terms[term] = [pattern]

# 변환 규칙 딕셔너리 생성
transformations = terms.copy()

# 변환된 단어를 기록하기 위한 딕셔너리
changes = {}

# 변환 함수 정의
def replace_and_log(original_text, pattern, replacement, changes_dict):
    # 변환 후 단어는 그대로 사용
    modified_replacement = replacement
    
    # 패턴에 매칭되는 모든 부분을 찾아서
    matches = re.findall(pattern, original_text)
    # 매칭된 부분이 있다면 변환하고 딕셔너리에 추가
    if matches:
        for match in matches:
            # 변환 내역 기록 (여러 번 나올 수 있어도 모두 기록)
            if match not in changes_dict:
                changes_dict[match] = modified_replacement
        # 텍스트에서 패턴을 변환
        modified_text = re.sub(pattern, modified_replacement, original_text)
        return modified_text
    return original_text

# 변환 대상 컬럼 설정
target_columns = ['title', 'sub-title', 'content']

# 모든 엑셀 파일을 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_excel(file_path)  # 엑셀 파일 읽기

        # 지정된 컬럼만 변환 진행
        for column in target_columns:
            if column in df.columns:
                print(f"  Processing column: {column}")
                for replacement, patterns in transformations.items():
                    for pattern in patterns:
                        df[column] = df[column].apply(
                            lambda x: replace_and_log(x, pattern, replacement, changes) if isinstance(x, str) else x
                        )
        
        # 변환된 데이터 저장
        output_path = os.path.join(folder_path, f'1_{filename}')
        df.to_excel(output_path, index=False)  # 변환된 파일을 새로운 이름으로 저장

# 변환된 단어 목록 출력
print("\n변환된 단어 목록:")
for original, transformed in changes.items():
    print(f"'{original}' -> '{transformed}'")


In [ ]:
#2매체명: 《매체명》
import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'

# 변환할 단어 목록 (변환 후 단어 : [변환 전 패턴들])
special_chars = ['＇', r'\"', r'\(', r'\)', r'\[', r'\]', r'\{', r'\}', r'\<', r'\>', '《', '》', '｢', '｣', '󰡔', '󰡕', '【', '】']

# 변환 후 단어들 리스트 및 변환 전 단어 규칙
terms = {
    "가톨릭신문": [r"가\s*톨\s*릭\s*시\s*보", r"가\s*톨\s*릭\s*신\s*보"],
    "문학과지성": [r"문\s*지"],
    "숙란": [r"숙\s*명\s*여\s*고\s*학\s*보"],
    "신작15인집": [r"신\s*작\s*십\s*오\s*인\s*집"],
    "여상": [r"女\s*像"],
}

# 추가 작품명들 변환 규칙 생성 (기존 규칙)
additional_terms = [
    "가정생활", "가톨릭신문", "경향신문", "국제신보", "동아일보", "문학과지성", "문학사상", "문화일보", "민주신보", "부산일보",
    "사상계", "새벗", "세계의_문학", "세대", "소설문학", "숙란", "신동아", "신작15인집", "신태양", "여상",
    "여성동아", "여원", "월간경항", "월간문학", "월간중앙", "자유공론", "잡지", "전남일보", "전망", "정경문화",
    "조선일보", "주부생활", "중앙일보", "한국문학", "한국일보", "한국평론", "현대문학",
]

# 추가 작품명들에 대해 기본 패턴 생성
for term in additional_terms:
    terms[term] = [r"\s*".join(term.replace('_', ''))]

# 변환 규칙 딕셔너리 생성
transformations = {}
for replacement, patterns in terms.items():
    full_patterns = []
    for pattern in patterns:
        for char1 in special_chars:
            for char2 in special_chars:
                full_patterns.append(rf"{char1}\s*{pattern}\s*{char2}")
    transformations[replacement] = full_patterns

# 변환된 단어를 기록하기 위한 딕셔너리
changes = {}

# 변환 함수 정의
def replace_and_log(original_text, pattern, replacement, changes_dict):
    # 변환 후 단어에 <>를 씌워줌
    modified_replacement = f"《{replacement}》"
    
    # 패턴에 매칭되는 모든 부분을 찾아서
    matches = re.findall(pattern, original_text)
    # 매칭된 부분이 있다면 변환하고 딕셔너리에 추가
    if matches:
        for match in matches:
            changes_dict[match] = modified_replacement
        # 텍스트에서 패턴을 변환
        modified_text = re.sub(pattern, modified_replacement, original_text)
        return modified_text
    return original_text

# 변환 대상 컬럼 설정
target_columns = ['title', 'sub-title', 'content']

# 모든 엑셀 파일을 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_excel(file_path)  # 엑셀 파일 읽기

        # 지정된 컬럼만 변환 진행
        for column in target_columns:
            if column in df.columns:
                print(f"  Processing column: {column}")
                for replacement, patterns in transformations.items():
                    for pattern in patterns:
                        df[column] = df[column].apply(
                            lambda x: replace_and_log(x, pattern, replacement, changes) if isinstance(x, str) else x
                        )
        
        # 변환된 데이터 저장
        output_path = os.path.join(folder_path, f'2_{filename}')
        df.to_excel(output_path, index=False)  # 변환된 파일을 새로운 이름으로 저장
        print(f"  Saved processed file: {output_path}")

# 변환된 단어 목록 출력
print("\n변환된 단어 목록:")
for original, transformed in changes.items():
    print(f"'{original}' -> '{transformed}'")


In [ ]:
#3기관명

import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'

# 변환할 단어 목록 (변환 후 단어 : [변환 전 패턴들])
terms = {
    "박경리문학관": [r"박\s*경\s*리\s*문\s*화\s*관"],
    "서울대학교": [r"서\s*울\s*대\s*학", r"서\s*울\s*대"],
    "연세대학교": [r"연\s*세\s*대\s*학", r"연\s*세\s*대", r"연\s*대"],
    "원주": [r"원\s*주\s*시", r"원\s*주"],
    "이화여자대학교": [r"이\s*화\s*여\s*자\s*대\s*학", r"이\s*화\s*여\s*대", r"이\s*대"],
    "일본제국": [r"일\s*제"],
    "정릉동": [r"정\s*릉"],
    "진주여자고등학교": [r"진\s*주\s*여\s*고"],
    "충무": [r"충\s*무\s*시", r"충\s*무"],
    "토지문학제": [r"토\s*지\s*문\s*화\s*제"],
    "하동": [r"하\s*동\s*군", r"하\s*동"],
    "통영": [r"통\s*영\s*시", r"통\s*영"],
    "한국여류문학인회": [
        r"한\s*국\s*여\s*류\s*문\s*학\s*인\s*회", r"한\s*국\s*여\s*류\s*문\s*인\s*협\s*회",
        r"여\s*류\s*문\s*학\s*인\s*會", r"여\s*류\s*문\s*학\s*인\s*회"
    ],
    "현대문학상": [r"현\s*대\s*문\s*학\s*신\s*인\s*문\s*학\s*상", r"현\s*대\s*문\s*학\s*상"],
    "호암예술상": [r"호\s*함\s*상\s*예\s*술\s*상", r"호\s*암\s*예\s*술\s*상"]
}

# 추가 명들 변환 규칙 생성 (기존 규칙)
additional_terms = [
    "금관문화훈장", "내성문학상", "대한교과서", "동광출판사", "동민문화사",
    "만주", "민음사", "박경리_기념관", "박경리문학공원", "박경리문학관",
    "박경리문학상", "박경리문학제", "범우사", "보관문화훈장", "삼성출판사",
    "서문당", "서울대학교", "소설토지학교", "솔출판사", "수문서관",
    "신태양사", "연세대학교", "원주", "월탄문학상", "을유문화사",
    "이화여자대학교", "인촌상", "일본제국", "일월서각", "정릉동",
    "지리산", "지식산업사", "진주여자고등학교", "충무", "토지문학제",
    "토지문화관", "토지문화재단", "토지사랑회", "통영", "평사리",
    "하동", "한국여류문학상", "한국여류문학인회", "현대문학사", "현대문학상",
    "현암사", "호암예술상", "환경운동연합"
]

# 추가 명들에 대해 기본 패턴 생성
for term in additional_terms:
    if term not in terms:
        pattern = rf"{r'\s*'.join(term)}"
        terms[term] = [pattern]

# 변환 규칙 딕셔너리 생성
transformations = terms.copy()

# 변환된 단어를 기록하기 위한 딕셔너리
changes = {}

# 변환 함수 정의
def replace_and_log(original_text, pattern, replacement, changes_dict):
    # 변환 후 단어는 그대로 사용
    modified_replacement = replacement
    
    # 패턴에 매칭되는 모든 부분을 찾아서
    matches = re.findall(pattern, original_text)
    # 매칭된 부분이 있다면 변환하고 딕셔너리에 추가
    if matches:
        for match in matches:
            # 변환 내역 기록 (여러 번 나올 수 있어도 모두 기록)
            if match not in changes_dict:
                changes_dict[match] = modified_replacement
        # 텍스트에서 패턴을 변환
        modified_text = re.sub(pattern, modified_replacement, original_text)
        return modified_text
    return original_text

# 변환 대상 컬럼 설정
target_columns = ['title', 'sub-title', 'content']

# 모든 엑셀 파일을 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_excel(file_path)  # 엑셀 파일 읽기

        # 지정된 컬럼만 변환 진행
        for column in target_columns:
            if column in df.columns:
                print(f"  Processing column: {column}")
                for replacement, patterns in transformations.items():
                    for pattern in patterns:
                        df[column] = df[column].apply(
                            lambda x: replace_and_log(x, pattern, replacement, changes) if isinstance(x, str) else x
                        )
        
        # 변환된 데이터 저장
        output_path = os.path.join(folder_path, f'3_{filename}')
        df.to_excel(output_path, index=False)  # 변환된 파일을 새로운 이름으로 저장

# 변환된 단어 목록 출력
print("\n변환된 단어 목록:")
for original, transformed in changes.items():
    print(f"'{original}' -> '{transformed}'")


In [ ]:
#4국가 5장르 6기타 7호칭

import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'

# 변환할 단어 목록 (변환 후 단어 : [변환 전 패턴들])
terms = {
    "박경리_작": [r"박\s*경\s*리\s*작", r"박\s*경\s*리\s*作"],
    "박경리_저": [r"박\s*경\s*리\s*저", r"박\s*경\s*리\s*著"],
    "여류": [r"여\s*류", r"녀\s*류"],
    "여성": [r"여\s*성", r"녀\s*성"],
    "여학생": [r"여\s*학\s*생", r"녀\s*학\s*생"],
    "대학생": [r"대\s*학\s*生"],
    "베스트셀러" : [r"베\s*스\s*트\s*셀\s*러", r"베\s*스\s*트\s*·\s*셀\s*러", r"베\s*슷\s*트\s*셀\s*러"]
    

}

# 추가 명들 변환 규칙 생성 (기존 규칙)
additional_terms = [
    "중국", "일본",
    '장편소설', '단편소설', '대하소설', '연재소설', '역사소설', '문예영화', '영화', '동화', '문단', '문학',
    '문화', '소설', '수필', '원작', '작품', '저자',
    '명예문학박사', '민족주의자', '민족주의', '일본인', '극일', '기념', '대중', '문명', '문인', '반일',
    '수상', '신예', '일본론', '중일',
    '소설가', '여사', '선생', '박경리씨', '작가',
]

# 추가 명들에 대해 기본 패턴 생성
for term in additional_terms:
    if term not in terms:
        pattern = rf"{r'\s*'.join(term)}"
        terms[term] = [pattern]

# 변환 규칙 딕셔너리 생성
transformations = terms.copy()

# 변환된 단어를 기록하기 위한 딕셔너리
changes = {}

# 변환 함수 정의
def replace_and_log(original_text, pattern, replacement, changes_dict):
    # 변환 후 단어는 그대로 사용
    modified_replacement = replacement
    
    # 패턴에 매칭되는 모든 부분을 찾아서
    matches = re.findall(pattern, original_text)
    # 매칭된 부분이 있다면 변환하고 딕셔너리에 추가
    if matches:
        for match in matches:
            # 변환 내역 기록 (여러 번 나올 수 있어도 모두 기록)
            if match not in changes_dict:
                changes_dict[match] = modified_replacement
        # 텍스트에서 패턴을 변환
        modified_text = re.sub(pattern, modified_replacement, original_text)
        return modified_text
    return original_text

# 변환 대상 컬럼 설정
target_columns = ['title', 'sub-title', 'content']

# 모든 엑셀 파일을 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_excel(file_path)  # 엑셀 파일 읽기

        # 지정된 컬럼만 변환 진행
        for column in target_columns:
            if column in df.columns:
                print(f"  Processing column: {column}")
                for replacement, patterns in transformations.items():
                    for pattern in patterns:
                        df[column] = df[column].apply(
                            lambda x: replace_and_log(x, pattern, replacement, changes) if isinstance(x, str) else x
                        )
        
        # 변환된 데이터 저장
        output_path = os.path.join(folder_path, f'456_{filename}')
        df.to_excel(output_path, index=False)  # 변환된 파일을 새로운 이름으로 저장

# 변환된 단어 목록 출력
print("\n변환된 단어 목록:")
for original, transformed in changes.items():
    print(f"'{original}' -> '{transformed}'")
